In [6]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("./Data/ADANIENT.csv")

df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = df['Date'].str.split(' ').str[0]
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.dropna()

In [18]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2024-01-02,2928.35,2947.00,2842.00,2932.05,130160
2024-01-03,3049.00,3199.45,2957.05,3003.95,1114801
2024-01-04,3034.95,3044.75,2991.05,2998.60,310738
2024-01-05,3000.00,3064.15,2982.00,3006.70,369900
2024-01-08,3016.95,3024.45,2952.85,2963.55,187111
...,...,...,...,...,...
2025-07-11,2573.30,2590.00,2552.35,2562.35,6641
2025-07-14,2553.10,2604.00,2542.75,2583.50,42718
2025-07-15,2576.10,2605.50,2575.40,2600.05,36060


In [19]:
def calculate_dema(series, span):
    ema = series.ewm(span=span, adjust=False).mean()
    dema = 2 * ema - ema.ewm(span=span, adjust=False).mean()
    return dema

In [20]:
df['DEMA_20'] = calculate_dema(df['Close'], 20)
df['DEMA_30'] = calculate_dema(df['Close'], 30)

In [21]:
df['Signal'] = 'Hold'

# Buy signal: DEMA_20 crosses above DEMA_50
df.loc[
    (df['DEMA_20'] > df['DEMA_30']) & (df['DEMA_20'].shift(1) <= df['DEMA_30'].shift(1)),
    'Signal'
] = 'Buy'

# Sell signal: DEMA_20 crosses below DEMA_50
df.loc[
    (df['DEMA_20'] < df['DEMA_30']) & (df['DEMA_20'].shift(1) >= df['DEMA_30'].shift(1)),
    'Signal'
] = 'Sell'


In [22]:
# Initialize trade tracking
position = None
buy_price = 0
trades = []

for date, row in df.iterrows():
    if row['Signal'] == 'Buy' and position is None:
        buy_price = row['Close']
        position = 'Long'
        trades.append({'Date': date, 'Type': 'Buy', 'Price': buy_price})
    
    elif row['Signal'] == 'Sell' and position == 'Long':
        sell_price = row['Close']
        profit = sell_price - buy_price
        trades.append({'Date': date, 'Type': 'Sell', 'Price': sell_price, 'Profit': profit})
        position = None


In [23]:
trades_df = pd.DataFrame(trades)

total_profit = trades_df['Profit'].sum()
win_trades = trades_df[trades_df['Profit'] > 0]
loss_trades = trades_df[trades_df['Profit'] <= 0]
win_rate = len(win_trades) / (len(win_trades) + len(loss_trades)) * 100 if (len(win_trades) + len(loss_trades)) > 0 else 0

print("\n📈 Backtest Results:")
print(trades_df)
print(f"\nTotal Trades: {len(trades_df)//2}")
print(f"Total Profit: ₹{total_profit:.2f}")
print(f"Win Rate: {win_rate:.2f}%")


📈 Backtest Results:
         Date  Type    Price  Profit
0  2024-01-03   Buy  3003.95     NaN
1  2024-01-23  Sell  2897.25 -106.70
2  2024-01-30   Buy  3091.55     NaN
3  2024-03-11  Sell  3207.00  115.45
4  2024-05-21   Buy  3115.75     NaN
5  2024-06-26  Sell  3169.35   53.60
6  2024-08-02   Buy  3160.65     NaN
7  2024-08-05  Sell  3035.95 -124.70
8  2024-08-07   Buy  3185.25     NaN
9  2024-08-26  Sell  3068.90 -116.35
10 2024-09-24   Buy  3092.40     NaN
11 2024-10-21  Sell  2936.35 -156.05
12 2024-12-16   Buy  2511.75     NaN
13 2025-01-30  Sell  2252.65 -259.10
14 2025-02-05   Buy  2324.75     NaN
15 2025-02-19  Sell  2179.90 -144.85
16 2025-03-06   Buy  2253.15     NaN
17 2025-05-08  Sell  2285.50   32.35
18 2025-05-13   Buy  2440.90     NaN
19 2025-06-17  Sell  2487.90   47.00
20 2025-07-01   Buy  2620.20     NaN
21 2025-07-10  Sell  2579.70  -40.50

Total Trades: 11
Total Profit: ₹-699.85
Win Rate: 36.36%


In [24]:
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'browser'
fig = go.Figure()

# Candlestick chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Price',
    increasing_line_color='green',
    decreasing_line_color='red'
))

# DEMA 20
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['DEMA_20'],
    mode='lines',
    name='DEMA 20',
    line=dict(color='blue', width=1)
))

# DEMA 50
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['DEMA_30'],
    mode='lines',
    name='DEMA 30',
    line=dict(color='orange', width=1)
))

# Buy markers
buy_signals = df[df['Signal'] == 'Buy']
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['Low'] * 0.995,
    mode='markers',
    name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Sell markers
sell_signals = df[df['Signal'] == 'Sell']
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['High'] * 1.005,
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Final layout
fig.update_layout(
    title='SBILIFE - DEMA Strategy Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template='plotly_white',
    legend=dict(x=0.01, y=0.99)
)

fig.show()


In [25]:
number_of_shares = int(100000 / buy_price)


In [26]:
capital = 100000

position = None
buy_price = 0
shares = 0
trades = []

for date, row in df.iterrows():
    if row['Signal'] == 'Buy' and position is None:
        buy_price = row['Close']
        shares = int(capital / buy_price)
        position = 'Long'
        trades.append({
            'Date': date,
            'Type': 'Buy',
            'Price': buy_price,
            'Shares': shares
        })

    elif row['Signal'] == 'Sell' and position == 'Long':
        sell_price = row['Close']
        profit_per_share = sell_price - buy_price
        total_profit = profit_per_share * shares
        trades.append({
            'Date': date,
            'Type': 'Sell',
            'Price': sell_price,
            'Shares': shares,
            'Profit_per_share': profit_per_share,
            'Total_Profit': total_profit
        })
        position = None


In [27]:
trades_df = pd.DataFrame(trades)

sell_trades = trades_df[trades_df['Type'] == 'Sell']
total_profit = sell_trades['Total_Profit'].sum()
win_trades = sell_trades[sell_trades['Total_Profit'] > 0]
loss_trades = sell_trades[sell_trades['Total_Profit'] <= 0]
win_rate = len(win_trades) / (len(win_trades) + len(loss_trades)) * 100 if (len(win_trades) + len(loss_trades)) > 0 else 0

print("\n📈 Backtest Results (₹1 lakh capital):")
print(trades_df)
print(f"\nTotal Trades: {len(sell_trades)}")
print(f"Total Profit: ₹{total_profit:,.2f}")
print(f"Win Rate: {win_rate:.2f}%")



📈 Backtest Results (₹1 lakh capital):
         Date  Type    Price  Shares  Profit_per_share  Total_Profit
0  2024-01-03   Buy  3003.95      33               NaN           NaN
1  2024-01-23  Sell  2897.25      33           -106.70      -3521.10
2  2024-01-30   Buy  3091.55      32               NaN           NaN
3  2024-03-11  Sell  3207.00      32            115.45       3694.40
4  2024-05-21   Buy  3115.75      32               NaN           NaN
5  2024-06-26  Sell  3169.35      32             53.60       1715.20
6  2024-08-02   Buy  3160.65      31               NaN           NaN
7  2024-08-05  Sell  3035.95      31           -124.70      -3865.70
8  2024-08-07   Buy  3185.25      31               NaN           NaN
9  2024-08-26  Sell  3068.90      31           -116.35      -3606.85
10 2024-09-24   Buy  3092.40      32               NaN           NaN
11 2024-10-21  Sell  2936.35      32           -156.05      -4993.60
12 2024-12-16   Buy  2511.75      39               NaN          

In [28]:
df

,Open,High,Low,Close,Volume,DEMA_20,DEMA_30,Signal
Date,,,,,,,,
2024-01-02,2928.35,2947.00,2842.00,2932.05,130160,2932.050000,2932.050000,Hold
2024-01-03,3049.00,3199.45,2957.05,3003.95,1114801,2945.093084,2941.028148,Buy
2024-01-04,3034.95,3044.75,2991.05,2998.60,310738,2955.333404,2948.479042,Hold
2024-01-05,3000.00,3064.15,2982.00,3006.70,369900,2965.577897,2956.219611,Hold
2024-01-08,3016.95,3024.45,2952.85,2963.55,187111,2966.509305,2957.815594,Hold
...,...,...,...,...,...,...,...,...
2025-07-11,2573.30,2590.00,2552.35,2562.35,6641,2596.896386,2599.509527,Hold
2025-07-14,2553.10,2604.00,2542.75,2583.50,42718,2597.189387,2600.622987,Hold
2025-07-15,2576.10,2605.50,2575.40,2600.05,36060,2600.307336,2603.592736,Hold
